---

## EVCS POI Candidate Selection

This section outlines the process of selecting POI (Point of Interest) candidates categorized by region.

- The above approach represents the process of selecting POI candidates categorized by region. POI filtering was applied when selecting POI candidates for all regions.
  
  - Before filtering, OSM_POI data was imported, and POIs were extracted for each categorized region (Atlanta, Suburban, Rural) using GIS tools.
  
  - For each extracted POI candidate, filtering was performed using the 'fclass' column, which contains information about building usage, utilizing GIS tools.

<br>

---

### Summary

- **Level 2 POI Candidate Selection**: Georgia, using GIS tools & filtering.  
- **DCFC POI Candidate Selection**: Georgia, using GIS tools & filtering.  
<br>
---


In [ ]:
gpkg_file_path = "Path To your Rural Region POI"
output_path =  "rural_poi_candidate.gpkg"
region = 'rural'

def process_poi_data(region, gpkg_file_path, output_path):
    # Define file paths and fclass categories based on the region
    if region == 'rural':
        selected_fclasses = ['supermarket', 'school', 'library', 'bank', 'fast_food', 'restaurant', 'museum', 
                             'college', 'hospital', 'university', 'town_hall', 'stadium', 'courthouse', 
                             'public_building', 'hotel', 'pharmacy', 'supermarket', 'department_store', 
                             'gift_shop', 'car_rental', 'zoo', 'motel', 'mall', 'camp_site', 'general']  # Rural-specific categories
    # Read the GPKG file
    poi_gdf = gpd.read_file(gpkg_file_path)

    # Filter the POI data based on the selected fclass categories
    selected_poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_fclasses)]
    
    # Get the count of filtered features
    selected_feature_count = selected_poi_gdf.shape[0]
    print(f"Selected feature count for {region}: {selected_feature_count}")

    # Save the filtered data to a new GPKG file
    selected_poi_gdf.to_file(output_path, driver='GPKG')
    print(f"Filtered POI data saved for {region} at {output_path}")

# Example usage:
process_poi_data(region, gpkg_file_path, output_path ) 


## Case 5: Rural Level 2 EVCS Charger Location Selection  
## Case 6: Rural DCFC EVCS Charger Location Selection

For **Case 5** and **Case 6**, which pertain to **Rural areas**, the demand for fast charging (DCFC) is typically low. Most charging needs in rural areas are met with slower chargers (Level 2), and thus, the allocated **capacity** for EVCS in these regions is not high.

When installing **EVCS** and **DCFC** chargers in rural areas, a thorough **POI filtering process** was conducted, followed by the use of the **Demand Map** to determine the optimal locations for charger deployment. This ensured that chargers were placed where they would meet the actual demand of the rural population while making efficient use of available capacity.

### Algorithm Summary:

1. **Load Data**: Load rural polygons, POI candidate data, and demand map (GeoTIFF).

2. **Convert Demand Pixels to Polygons**: Extract demand values from the GeoTIFF file and convert them into polygons.

3. **Spatial Join**: Perform a spatial join between the POI data and demand polygons to assign demand values to POIs.

4. **POI Selection**:
   - Filter POIs within each rural polygon.
   - Select the POI with the highest demand, prioritizing those in **priority categories**.
   - If no priority category POI exists, randomly select the highest demand POI.

5. **DCFC & Level2 Type Are Same Algorithm in Here** :

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import box
import random

# Define priority POI categories
selected_category = ['department_store', 'town_hall', 'hospital', 'market_place', 
                     'mall', 'public_building']

# File paths
poi_path = 'Path to your rural_poi_candidate.gpkg'  # POI# Path to POI candidate data
polygon_path = "Path to your Rural Polygon.gpkg"  # Path to rural polygons
demand_tif_path = 'Path to your Demand map'
output_path = "Path to your Output Path"  # Output directory

# 1. Load target region polygons and POI data
polygon_gdf = gpd.read_file(polygon_path)  # Load rural county polygons
poi_gdf = gpd.read_file(poi_path)  # Load POI candidate data

# 2. Read the demand data from the tif file and convert demand pixels to polygons
with rasterio.open(demand_tif_path) as src:
    affine = src.transform
    demand_data = src.read(1)  # Read the first band (demand values)
    rows, cols = np.where(demand_data > 0)  # Select only pixels with demand values
    
    demand_polygons = []
    demand_values = []
    
    for row, col in zip(rows, cols):
        # Convert pixel coordinates to geographic coordinates (bottom-left and top-right)
        x_min, y_min = affine * (col, row)
        x_max, y_max = affine * (col + 1, row + 1)
        demand_polygons.append(box(x_min, y_min, x_max, y_max))  # Convert the pixel to a polygon
        demand_values.append(demand_data[row, col])  # Store the demand value

# 3. Convert demand pixels into a GeoDataFrame
demand_gdf = gpd.GeoDataFrame({'demand': demand_values, 'geometry': demand_polygons}, crs="EPSG:3857")

# 4. Perform a spatial join to map demand values to POIs
poi_with_demand = gpd.sjoin(poi_gdf, demand_gdf, how='left', op='within')

# Select the POI with the highest demand in each polygon
selected_pois = []
for index, polygon in polygon_gdf.iterrows():
    # Filter POIs within the current polygon (POIs with mapped demand values)
    contained_poi = poi_with_demand[poi_with_demand.within(polygon.geometry)]

    if contained_poi.empty:
        continue  # Skip if no POIs are found in the polygon
    
    # 1) Filter POIs with the highest demand value
    max_demand_value = contained_poi['demand'].max()
    highest_demand_pois = contained_poi[contained_poi['demand'] == max_demand_value]
    
    if highest_demand_pois.empty:
        continue  # Skip if no POI has a demand value
    
    # 2) Search for POIs belonging to the preferred categories
    preferred_pois = highest_demand_pois[highest_demand_pois['fclass'].isin(selected_category)]
    
    if not preferred_pois.empty:
        # Randomly select a POI from the preferred category
        selected_poi = preferred_pois.sample()
    else:
        # If no POI belongs to the preferred category, randomly select from the highest demand POIs
        selected_poi = highest_demand_pois.sample()
    
    # Store the selected POI
    selected_pois.append(selected_poi)

# Save the selected POIs to a GeoDataFrame
if selected_pois:
    selected_pois_gdf = gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs='EPSG:3857')
    
    # Truncate column names to 10 characters (to comply with ESRI Shapefile column name limits)
    selected_pois_gdf.columns = [str(col)[:10] for col in selected_pois_gdf.columns]
    
    # Save the result as a .gpkg file
    selected_pois_gdf.to_file(f"{output_path}rural_dcfc_evcs.gpkg", driver="GPKG")
    
else:
    print("No POIs were selected.")  # Print a message if no POIs were selected
